In [1]:
import pandas as pd
import pyreadr
import spacy
import numpy as np

In [2]:
#read in the data
d = pyreadr.read_r('Downloads/narrative_submissions.RDS')
df = d[None] #make into pandas df
df.sample(10)

,author,created_utc,id,num_comments,score,selftext,subreddit_subscribers,title
437,Beelzebubbin,1531498034,8yl8kf,18,7,"About two years ago, just after the election, ...",4187,Walking away was the best decision of my life.
569,Rand_Omname,1532890045,92wzos,23,37,Be honest.\n\nI walked away in 2014 or 2015 wh...,5341,Posters on this sub: when did you #walkaway?
559,dankbonkripper78_,1532701208,92cx1e,47,74,I know this subreddit is oftener viewed as con...,5290,What is the walk away movement?
265,UnperformedKubrick,1531275957,8xvpt1,1,0,I am also top scientist of USA and have had se...,1923,I was very liberal and believed in lots of lib...
398,ninjarager,1531417305,8ybxt7,0,0,I called the democratic party home for so many...,4064,My story.
368,The_Bug_Lord,1531350378,8y4eqq,0,1,ALERT! RUSSIAN SPY DETECTED!,3852,"OMG, HE'S A RUSSIAN SPY"
425,temujin1234,1531465683,8yhu4n,23,0,Walk towards an autocrat. No thanks.\n\nWould...,4143,"C'mon, you all really mean walktowards."
538,k3wlmeme,1532478163,91myge,59,83,"I've been a very good friend to many. Loyal, t...",5135,A showerthought I had about coming out as a co...
546,wherethewhisperslie,1532556578,91we7e,10,3,So if this sub is supposed to be for democrats...,5206,Question about open discussion
619,DeplorableCadet,1533963804,96e484,0,1,Patriots are walking away from the Democrat pa...,5842,https://youtu.be/-GesMTNxjLI


In [3]:
nlp = spacy.load('en_core_web_lg')

In [4]:
#run all narratives through the spacy parsing paradigm

df['parsed_narr'] = list(nlp.pipe(df['selftext']))

In [5]:
for token in df['parsed_narr'][6]:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])

Hello ROOT Hello INTJ [everyone, .]
everyone npadvmod Hello INTJ []
. punct Hello INTJ []
I nsubj been AUX []
've aux been AUX []
been ROOT been AUX [I, 've, fascinated, learned, ,, reflects, .]
super advmod fascinated ADJ []
fascinated acomp been AUX [super, by]
by prep fascinated ADJ [movement]
this det movement NOUN []
movement pobj by ADP [this]
since mark learned VERB []
I nsubj learned VERB []
learned advcl been AUX [since, I, about]
about prep learned VERB [it]
it pobj about ADP []
, punct been AUX []
because mark reflects VERB []
it nsubj reflects VERB []
reflects advcl been AUX [because, it, shift]
the det shift NOUN []
subtle amod shift NOUN []
shift dobj reflects VERB [the, subtle, in, having]
in prep shift NOUN [ideology]
ideology pobj in ADP []
I nsubj having VERB []
've aux having VERB []
been aux having VERB []
having relcl shift NOUN [I, 've, been, over]
over prep having VERB [months]
the det months NOUN []
past amod months NOUN []
several amod months NOUN []
months pob

caught acl case NOUN [I, get, up, in]
up prt caught VERB []
in prep caught VERB [storm]
a det storm NOUN []
social amod media NOUN []
media compound storm NOUN [social]
storm pobj in ADP [a, media]
? punct are AUX [

]


  ? PUNCT []
* ROOT * PUNCT []
* ROOT * PUNCT []
TL;DR ROOT TL;DR PROPN []
* ROOT * PUNCT []
* ROOT * PUNCT []
If mark talk VERB []
I nsubj talk VERB []
talk advcl lose VERB [If, I, on, about]
on prep talk VERB [Twitter]
Twitter pobj on ADP []
about prep talk VERB [views]
my poss views NOUN []
no det views NOUN []
- punct leftist ADJ []
longer advmod leftist ADJ []
- punct leftist ADJ []
leftist amod views NOUN [-, longer, -]
views pobj about ADP [my, no, leftist]
, punct lose VERB []
I nsubj lose VERB []
could aux lose VERB []
lose ROOT lose VERB [talk, ,, I, could, standing, in]
my poss standing NOUN []
standing dobj lose VERB [my]
in prep lose VERB [community]
a det community NOUN []
community pobj in ADP [a, 's]
that nsubj 's AUX []
's relcl community NOUN [that, i

In [6]:
spacy.displacy.render(df['parsed_narr'][6], style='dep')

In [102]:
import spacy
from spacy.symbols import nsubj, VERB

doc = df['parsed_narr'][6]

# Finding a verb with a subject from below — good
verbs = set()
for possible_subject in doc:
    if possible_subject.dep == nsubj and possible_subject.head.pos == VERB:
        verbs.add(possible_subject.head)
print(verbs)

{seen, become, prepare, thought, stand, trying, happening, leaned, read, disconnecting, happens, talk, written, lose, prepare, know, having, reflects, wondering, undo, handle, learned, felt, stood}


In [114]:
verbs = list()
tense = list()
ids = list()

i = 0 
for d in df['parsed_narr']:
    for possible_verb in d:
        if possible_verb.pos == VERB:
            verbs.append(possible_verb)
            tense.append(possible_verb.tag_)
            ids.append(df['id'][i])
    i += 1

In [117]:
verbs_in_submissions = pd.DataFrame(
    {'verbs': verbs,
     'tense': tense,
     'postid': ids
    })

In [119]:
verbs_in_submissions.sample(10)

,verbs,tense,postid
4959,told,VBD,8xzz88
14024,watching,VBG,9hh75e
6442,seems,VBZ,8yflbk
9896,getting,VBG,90y7fa
11681,causing,VBG,9540ai
14592,met,VBD,9jss5w
3663,feel,VB,8xtb76
3186,fucked,VBD,8xrspr
13323,disregarded,VBN,9gyoys
2051,'d,MD,8xpkmj


In [121]:
verbs_in_submissions.to_csv("all_verbs_new.csv")

In [19]:
doc = df['parsed_narr'][6]

noun_phrases = list()
post_ids = list()
i = 0

for d in df['parsed_narr']:
    for phrase in d.noun_chunks:
        noun_phrases.append(phrase)
        post_ids.append(df['id'][i])
    i += 1

In [20]:
noun_phrases = pd.DataFrame({"noun_phrase": noun_phrases, "postid": post_ids})

In [26]:
noun_phrases.head()

noun_phrases.to_csv("noun_phrases.csv")

In [12]:
def noun_verb_pairs(doc):
    nounverbs = list()
    for tok in doc:
        if tok.dep_ == 'ROOT':
            nounverbs.append((child(tok,'nsubj'),tok,child(tok,'dobj')))
    return nounverbs

def child(tok, dep): # helper function
    for c in tok.children:
        if c.dep_== dep:
            return c
    return None

noun_verb_pairs(doc)

[(None, Hello, None),
 (I, been, None),
 (I, was, None),
 (I, written, None),
 (None, because, None),
 (it, 's, None),
 (I, stand, None),
 (I, have, platform),
 (I, disconnecting, None),
 (I, seen, None),
 (None, TORN, members),
 (None, of, None),
 (None, made, None),
 (community, been, None),
 (None, hurt, None),
 (None, lot, None),
 (None, *, None),
 (they, have, power),
 (I, trying, None),
 (community, undo, work),
 (None, Thank, you),
 (anyone, stood, None),
 (you, handle, that),
 (What, are, None),
 (None, *, None),
 (None, *, None),
 (None, TL;DR, None),
 (None, *, None),
 (None, *, None),
 (I, lose, standing),
 (None, destroyed, None),
 (I, prepare, myself)]

In [22]:
nvpairs = list()

for d in df['parsed_narr']:
    nvpairs.append(noun_verb_pairs(d))

,0,1,2,3,4,5,6,7,8,9,...,297,298,299,300,301,302,303,304,305,306
0,"(sub, Is, None)",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,"(None, Hello, None)","(I, sent, message)",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,"(None, is, None)","(None, updated, None)",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,"(We, added, flairs)","(None, Pick, yours)","(None, Comment, None)",None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,"(None, Welcome, None)","(you, are, None)","(None, are, None)","(You, have, None)","(You, have, None)","(You, despise, man)","(you, have, None)","(left, aligns, None)","(This, is, None)","(None, because, None)",...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
788,"(None, deleted, None)",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
789,"(None, deleted, None)",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
790,"(None, deleted, None)",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
791,"(We, killed, actor)","(Hollywood, be, None)",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


Getting all subject, verb, direct object pairs 

In [18]:
#all possible subjects and objects
SUBJECTS = ["nsubj", "nsubjpass", "csubj", "csubjpass", "agent", "expl"]
OBJECTS = ["dobj", "dative", "attr", "oprd"]